In [28]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import OneHotEncoder

# modeling
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import *
from sklearn.linear_model import LogisticRegression
import sklearn.metrics
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import fbeta_score, make_scorer


#explainability
import shap

#serialization
import joblib

#remove warnings
import warnings 
warnings.filterwarnings("ignore")

In [29]:
#read the data 
app_train = pd.read_csv("../raw_data/application_train.csv").astype("object")

In [30]:
X_train, X_test, y_train, y_test = train_test_split(app_train.drop(['TARGET'], axis=1), #x
                                                    app_train['TARGET'], #y
                                                    test_size=0.2,
                                                    stratify=app_train['TARGET'], #stratified sample
                                                    random_state=123)

# Features Types

We need to convert variables that are not correctly cast as of float in the raw data : 

In [31]:
def convert_dtypes_func(df):
    #List the columns to be converted to float
    list_columns_to_convert_to_float = ("CNT_CHILDREN" 
                                ,"AMT_INCOME_TOTAL" 
                                ,"AMT_CREDIT" 
                                ,"AMT_ANNUITY" 
                                ,"AMT_GOODS_PRICE" 
                                ,"DAYS_BIRTH"
                                ,"DAYS_EMPLOYED" 
                                ,"DAYS_REGISTRATION"
                                ,"REGION_RATING_CLIENT"
                                ,"REGION_POPULATION_RELATIVE"
                                ,"HOUR_APPR_PROCESS_START"
                                ,"CNT_FAM_MEMBERS" 
                                ,"DAYS_ID_PUBLISH"
                                ,"OWN_CAR_AGE"
                                ,"EXT_SOURCE_1               "
                                ,"EXT_SOURCE_2               "
                                ,"EXT_SOURCE_3               "
                                ,"APARTMENTS_AVG             "
                                ,"BASEMENTAREA_AVG           "
                                ,"YEARS_BEGINEXPLUATATION_AVG"
                                ,"YEARS_BUILD_AVG            "
                                ,"COMMONAREA_AVG             "
                                ,"ELEVATORS_AVG              "
                                ,"ENTRANCES_AVG              "
                                ,"FLOORSMAX_AVG              "
                                ,"FLOORSMIN_AVG              "
                                ,"LANDAREA_AVG               "
                                ,"LIVINGAPARTMENTS_AVG       "
                                ,"LIVINGAREA_AVG             "
                                ,"NONLIVINGAPARTMENTS_AVG    "
                                ,"NONLIVINGAREA_AVG          "
                                ,"APARTMENTS_MODE            "
                                ,"BASEMENTAREA_MODE          "
                                ,"YEARS_BEGINEXPLUATATION_MODE"
                                ,"YEARS_BUILD_MODE           "
                                ,"COMMONAREA_MODE            "
                                ,"ELEVATORS_MODE             "
                                ,"ENTRANCES_MODE             "
                                ,"FLOORSMAX_MODE             "
                                ,"FLOORSMIN_MODE             "
                                ,"LANDAREA_MODE              "
                                ,"LIVINGAPARTMENTS_MODE      "
                                ,"LIVINGAREA_MODE            "
                                ,"NONLIVINGAPARTMENTS_MODE   "
                                ,"NONLIVINGAREA_MODE         "
                                ,"APARTMENTS_MEDI            "
                                ,"BASEMENTAREA_MEDI          "
                                ,"YEARS_BEGINEXPLUATATION_MEDI"
                                ,"YEARS_BUILD_MEDI           "
                                ,"COMMONAREA_MEDI            "
                                ,"ELEVATORS_MEDI             "
                                ,"ENTRANCES_MEDI             "
                                ,"FLOORSMAX_MEDI             "
                                ,"FLOORSMIN_MEDI             "
                                ,"LANDAREA_MEDI              "
                                ,"LIVINGAPARTMENTS_MEDI      "
                                ,"LIVINGAREA_MEDI            "
                                ,"NONLIVINGAPARTMENTS_MEDI   "
                                ,"NONLIVINGAREA_MEDI         "
                                ,"TOTALAREA_MODE             "
                                ,"OBS_30_CNT_SOCIAL_CIRCLE   "
                                ,"DEF_30_CNT_SOCIAL_CIRCLE   "
                                ,"OBS_60_CNT_SOCIAL_CIRCLE   "
                                ,"DEF_60_CNT_SOCIAL_CIRCLE   "
                                ,"DAYS_LAST_PHONE_CHANGE     "
                                ,"AMT_REQ_CREDIT_BUREAU_HOUR"
                                ,"AMT_REQ_CREDIT_BUREAU_DAY "
                                ,"AMT_REQ_CREDIT_BUREAU_WEEK"
                                ,"AMT_REQ_CREDIT_BUREAU_MON "
                                ,"AMT_REQ_CREDIT_BUREAU_QRT "
                                ,"AMT_REQ_CREDIT_BUREAU_YEAR")       

    #remove spaces in the list created
    list_columns_to_convert_to_float = [s.strip() for s in list_columns_to_convert_to_float]       
    
    convert_count = 0

    #convert object columns to float
    for col in list_columns_to_convert_to_float:
        df[col] = df[col].astype(float)
        
        # Keep track of how many columns were label encoded
        convert_count += 1

    print('%d object columns were converted to int.' % convert_count)
    
    return df

In [32]:
X_train = convert_dtypes_func(X_train)

71 object columns were converted to int.


# Feature Engineering

In [33]:
def feature_eng(df):

    # Create an anomalous flag column
    df['DAYS_EMPLOYED_ANOM'] = df["DAYS_EMPLOYED"] == 365243

    # Replace the anomalous values with nan
    df['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

    #correct sign of Days Birth
    df["DAYS_BIRTH"] = abs(df["DAYS_BIRTH"])
    df['AGE_INT'] = round((df['DAYS_BIRTH'] / 365).astype(float),2)

    #footing financial ratios
    df['annuity_income_ratio'] = round((df['AMT_INCOME_TOTAL'] / df['AMT_ANNUITY']).astype(float),2)
    df['credit_annuity_ratio'] = round((df['AMT_CREDIT'] / df['AMT_ANNUITY']).astype(float),2)
    df['credit_goods_price_ratio'] = round((df['AMT_CREDIT'] / df['AMT_GOODS_PRICE']).astype(float),2)
    df['credit_downpayment'] = round((df['AMT_GOODS_PRICE'] - df['AMT_CREDIT']).astype(float),2)
 
    print('Feature engineering success')
   
    return df

In [34]:
X_train = feature_eng(X_train)

Feature engineering success


# Feature Selection

In [35]:
def df_feature_selection(df):
    columns_list = df.columns.to_list()

    columns_to_drop_intersec = (['DAYS_BIRTH',
    'REGION_RATING_CLIENT_W_CITY',
    'BASEMENTAREA_MODE',
    'YEARS_BUILD_MODE',
    'COMMONAREA_MODE',
    'ELEVATORS_MODE',
    'ENTRANCES_MODE',
    'FLOORSMAX_MODE',
    'LANDAREA_MODE',
    'LIVINGAPARTMENTS_MODE',
    'NONLIVINGAPARTMENTS_MODE',
    'NONLIVINGAREA_MODE',
    'APARTMENTS_MEDI',
    'BASEMENTAREA_MEDI',
    'YEARS_BEGINEXPLUATATION_MEDI',
    'YEARS_BUILD_MEDI',
    'COMMONAREA_MEDI',
    'ELEVATORS_MEDI',
    'ENTRANCES_MEDI',
    'FLOORSMAX_MEDI',
    'FLOORSMIN_MEDI',
    'LIVINGAPARTMENTS_MEDI',
    'LIVINGAREA_MEDI',
    'NONLIVINGAPARTMENTS_MEDI',
    'NONLIVINGAREA_MEDI',
    'OBS_60_CNT_SOCIAL_CIRCLE',
    "DAYS_EMPLOYED_ANOM"])

    #Drop columns
    df_selec_col = pd.DataFrame(data=df, columns=columns_list).drop(columns_to_drop_intersec, axis=1)
    
    print('Feature selection success')

    return df_selec_col

In [36]:
X_train = df_feature_selection(X_train)

Feature selection success


# Imputation - Encoding - Standardization - Imputation

In [37]:
#keep id into a separate serie
user_id = X_train[['SK_ID_CURR']]

#create df of features by type
cat_features = X_train.select_dtypes(include=['object']).drop(['SK_ID_CURR'], axis=1)
num_features = X_train.select_dtypes(exclude=['object'])

#imputation
categorical_imputer = SimpleImputer(missing_values=np.nan, strategy="most_frequent")
cat_features = categorical_imputer.fit_transform(cat_features)

simple_imputer = SimpleImputer(missing_values=np.nan, strategy="median")
num_features = simple_imputer.fit_transform(num_features)

#One hot encoding categorical variables
ohe = OneHotEncoder(handle_unknown='ignore')
cat_array = ohe.fit_transform(cat_features).todense()
cat_array = np.asarray(cat_array)

#Standard Scaling numerical variables
scaler = StandardScaler()
num_array = scaler.fit_transform(num_features)

#concatenate
X_train = np.concatenate([cat_array, num_array], axis=1)
# y = app_train['TARGET'].astype(int)

In [38]:
X_test = convert_dtypes_func(X_test)
X_test = feature_eng(X_test)
X_test = df_feature_selection(X_test)

#keep id into a separate serie
user_id = X_test[['SK_ID_CURR']]

#create df of features by type
cat_features = X_test.select_dtypes(include=['object']).drop(['SK_ID_CURR'], axis=1)
num_features = X_test.select_dtypes(exclude=['object'])

#imputation
cat_features = categorical_imputer.transform(cat_features)

num_features = simple_imputer.transform(num_features)

#One hot encoding categorical variables
cat_array = ohe.transform(cat_features).todense()
cat_array = np.asarray(cat_array)

#Standard Scaling numerical variables
num_array = scaler.transform(num_features)

#concatenate
X_test = np.concatenate([cat_array, num_array], axis=1)
# y = app_train['TARGET'].astype(int)

71 object columns were converted to int.
Feature engineering success
Feature selection success


In [ ]:
best_threshold_LogReg = 0.48

model_log = LogisticRegression(class_weight = "balanced")

tuned_parameters_ex = {"C": [0.005]}

df_results_Log_fin_ = train_test_fin(model_log,
                                    2,
                                    best_threshold_LogReg,
                                    tuned_parameters_ex,
                                    X_train,
                                    X_test,
                                    y_train,
                                    y_test)

In [50]:
scorer = make_scorer(fbeta_score, beta=2)

clf = GridSearchCV(model_log, 
                param_grid=tuned_parameters_ex, 
                scoring=scorer, 
                refit=True,
                cv=5,
                return_train_score=True
                )

In [51]:
clf

GridSearchCV(cv=5, estimator=LogisticRegression(class_weight='balanced'),
             param_grid={'C': [0.005]}, return_train_score=True,
             scoring=make_scorer(fbeta_score, beta=2))

In [39]:
def train_test_fin (model, beta, threshold, tuned_parameters, x_train, x_test, y_train, y_test, beta_search=None):
  
  #https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
  #https://openclassrooms.com/fr/courses/4297211-evaluez-les-performances-dun-modele-de-machine-learning/4308256-evaluez-un-algorithme-de-classification-qui-retourne-des-valeurs-binaires
  print(tuned_parameters)

  scorer = make_scorer(fbeta_score, beta=beta)

  clf = GridSearchCV(model, 
                    param_grid=tuned_parameters, 
                    scoring=scorer, 
                    refit=True,
                    cv=5,
                    return_train_score=True
                    )
  
  clf.fit(x_train, y_train)
  
  y_pred_train_proba = clf.best_estimator_.predict_proba(x_train) 
  y_pred_train_proba = y_pred_train_proba[:, 1]

  y_pred_proba = clf.best_estimator_.predict_proba(x_test) 
  y_pred_proba = y_pred_proba[:, 1]
  y_pred = y_pred_proba > threshold
  y_pred = y_pred.astype(int)

  #confusion matrix from predictions
  print(f'Confusion Matrix from predictions for {model} :')
  ConfusionMatrixDisplay.from_predictions(y_test, y_pred)
  plt.grid(False)
  plt.show()
  print()

  #performance metrics on test set
  recall_score = round(sklearn.metrics.recall_score(y_test, y_pred),4)
  f1_score = round(sklearn.metrics.f1_score(y_test, y_pred),4)
  roc_auc_score_train = round(sklearn.metrics.roc_auc_score(y_train, y_pred_train_proba),4)
  roc_auc_score = round(sklearn.metrics.roc_auc_score(y_test, y_pred_proba),4)
  fbeta_score_ = round(sklearn.metrics.fbeta_score(y_test, y_pred, beta=beta),4)

  #execution time
  mean_fit_time = clf.cv_results_['mean_fit_time']
  mean_score_time = clf.cv_results_['mean_score_time']
  n_splits = clf.n_splits_ #number of splits of training data
  n_iter = pd.DataFrame(clf.cv_results_).shape[0] #Iterations it takes per split for the solver to converge 
  execution_time = round(np.mean(mean_fit_time + mean_score_time) * n_splits * n_iter, 0)

  #return (i) perf metrics (ii) best hyperparams
  print(f'Best Threshold : {threshold} ')
  print(f'Performance Metrics for {model} :')
  print(f'Fbeta Score : {fbeta_score_}')
  print(f'Recall/sensitivity [true positive rate = TP / (TP + FN)]: {recall_score}')
  print(f'F1 Score : {f1_score}')
  print(f'AUROC : {roc_auc_score}')
  print(f'Execution time (sec): {execution_time}')
  print()

  print(f'Best Hyperparam(s) on training set :{clf.best_params_}')

  # store results in a dataframe
  df_results = pd.DataFrame(columns=['Model',
                                     'best_params_', 
                                     'roc_auc_score_train', 
                                     'roc_auc_score_test', 
                                     'recall_score_test',
                                     'fbeta_score_',
                                     'f1_score_test',
                                     'execution_time (sec)'
                                     ])

  data = {'Model': [model], 
          'best_params_' : [clf.best_params_], 
          'roc_auc_score_train' : [roc_auc_score_train],
          'roc_auc_score_test' : [roc_auc_score],
          'recall_score_test' : [recall_score], 
          'fbeta_score_' : [fbeta_score_], 
          'f1_score_test': [f1_score], 
          'execution_time (sec)': [execution_time]
          }

  results = pd.DataFrame.from_dict(data)

  df_results = df_results.append(results)
  
  if beta_search == True:

    beta_score_list = []
    for i in range(1,101):
        fbeta_results = fbeta_score(y_test,y_pred,beta=i)
        beta_score_list.append(fbeta_results)

    fig = plt.figure(figsize=(10,6))
    sns.lineplot(x=range(1,101), y=beta_score_list)
    plt.title('Fbeta Score with beta 0 to 100')

  return df_results

In [49]:
best_threshold_LogReg = 0.48

model_log = LogisticRegression(class_weight = "balanced")

tuned_parameters_ex = {"C": 0.005}

df_results_Log_fin_ = train_test_fin(model_log,
                                    2,
                                    best_threshold_LogReg,
                                    tuned_parameters_ex,
                                    X_train,
                                    X_test,
                                    y_train,
                                    y_test)

{'C': [0.005]}


NotFittedError: All estimators failed to fit

In [510]:
#Model with best params
model = LogisticRegression(class_weight = 'balanced', C=0.005, max_iter=1000)

#fit
model.fit(X_train,y_train)

LogisticRegression(C=0.005, class_weight='balanced', max_iter=1000)

In [511]:
#predict_proba returns the probability of the sample
#for each class in the model
#where classes are ordered as they are in self.classes_.
model.classes_

array([0, 1])

# SHAP

## Footing Shapley Explainer on Training Set

In [357]:
#cat columns list after ohe
cat_features_list = list(app_train.select_dtypes(include=['object']).drop(['SK_ID_CURR','TARGET'], axis=1).columns)
#https://stackoverflow.com/questions/54570947/feature-names-from-onehotencoder
cat_features_list_after_ohe = ohe.get_feature_names(cat_features_list).tolist()

#numerical columns list
num_features_list_after_preproc = list(app_train.select_dtypes(exclude=['object']).columns)

#concatenate list of features
features_list_after_prepr = cat_features_list_after_ohe + num_features_list_after_preproc

#transform X into a dataframe with column names
ohe_dataframe = pd.DataFrame(X, columns=features_list_after_prepr)

In [358]:
sub_sampled_train_data = shap.sample(ohe_dataframe, 50)
log_reg_explainer = shap.KernelExplainer(model.predict_proba, sub_sampled_train_data)

In [359]:
print(f'Shap Expected Value : {log_reg_explainer.expected_value}')

result_proba_training = model.predict_proba(X).mean(axis=0)
print(f'Model Mean Value (Theoretical Expected Value of default) : {result_proba_training[1]}')

Shap Expected Value : [0.51462926 0.48537074]
Model Mean Value (Theoretical Expected Value of default) : 0.4227130435028335


## Prediction on Test Data and Shap Values

In [360]:
# df_test = pd.read_csv("./dashboard_data/df_test.csv").astype(object)

# list_cat_features = ["NAME_CONTRACT_TYPE",
#     "CODE_GENDER",
#     "FLAG_OWN_CAR",
#     "FLAG_OWN_REALTY",
#     "NAME_TYPE_SUITE",
#     "NAME_INCOME_TYPE",
#     "NAME_EDUCATION_TYPE",
#     "NAME_FAMILY_STATUS",
#     "NAME_HOUSING_TYPE",
#     "FLAG_MOBIL",
#     "FLAG_EMP_PHONE",
#     "FLAG_WORK_PHONE",
#     "FLAG_CONT_MOBILE",
#     "FLAG_PHONE",
#     "FLAG_EMAIL",
#     "OCCUPATION_TYPE",
#     "WEEKDAY_APPR_PROCESS_START",
#     "REG_REGION_NOT_LIVE_REGION",
#     "REG_REGION_NOT_WORK_REGION",
#     "LIVE_REGION_NOT_WORK_REGION",
#     "REG_CITY_NOT_LIVE_CITY",
#     "REG_CITY_NOT_WORK_CITY",
#     "LIVE_CITY_NOT_WORK_CITY",
#     "ORGANIZATION_TYPE",
#     "FONDKAPREMONT_MODE",
#     "HOUSETYPE_MODE",
#     "WALLSMATERIAL_MODE",
#     "EMERGENCYSTATE_MODE",
#     "FLAG_DOCUMENT_2",
#     "FLAG_DOCUMENT_3",
#     "FLAG_DOCUMENT_4",
#     "FLAG_DOCUMENT_5",
#     "FLAG_DOCUMENT_6",
#     "FLAG_DOCUMENT_7",
#     "FLAG_DOCUMENT_8",
#     "FLAG_DOCUMENT_9",
#     "FLAG_DOCUMENT_10",
#     "FLAG_DOCUMENT_11",
#     "FLAG_DOCUMENT_12",
#     "FLAG_DOCUMENT_13",
#     "FLAG_DOCUMENT_14",
#     "FLAG_DOCUMENT_15",
#     "FLAG_DOCUMENT_16",
#     "FLAG_DOCUMENT_17",
#     "FLAG_DOCUMENT_18",
#     "FLAG_DOCUMENT_19",
#     "FLAG_DOCUMENT_20",
#     "FLAG_DOCUMENT_21"
# ]
# list_num_features = [
#     "CNT_CHILDREN",
#     "AMT_INCOME_TOTAL",
#     "AMT_CREDIT",
#     "AMT_ANNUITY",
#     "AMT_GOODS_PRICE",
#     "DAYS_EMPLOYED",
#     "DAYS_REGISTRATION",
#     "DAYS_ID_PUBLISH",
#     "REGION_RATING_CLIENT",
#     "REGION_POPULATION_RELATIVE",
#     "CNT_FAM_MEMBERS",
#     "HOUR_APPR_PROCESS_START",
#     "OWN_CAR_AGE",
#     "EXT_SOURCE_1",
#     "EXT_SOURCE_2",
#     "EXT_SOURCE_3",
#     "APARTMENTS_AVG",
#     "BASEMENTAREA_AVG",
#     "YEARS_BEGINEXPLUATATION_AVG",
#     "YEARS_BUILD_AVG",
#     "COMMONAREA_AVG",
#     "ELEVATORS_AVG",
#     "ENTRANCES_AVG",
#     "FLOORSMAX_AVG",
#     "FLOORSMIN_AVG",
#     "LANDAREA_AVG",
#     "LIVINGAPARTMENTS_AVG",
#     "LIVINGAREA_AVG",
#     "NONLIVINGAPARTMENTS_AVG",
#     "NONLIVINGAREA_AVG",
#     "APARTMENTS_MODE",
#     "YEARS_BEGINEXPLUATATION_MODE",
#     "FLOORSMIN_MODE",
#     "LIVINGAREA_MODE",
#     "LANDAREA_MEDI",
#     "TOTALAREA_MODE",
#     "OBS_30_CNT_SOCIAL_CIRCLE",
#     "DEF_30_CNT_SOCIAL_CIRCLE",
#     "DEF_60_CNT_SOCIAL_CIRCLE",
#     "DAYS_LAST_PHONE_CHANGE",
#     "AMT_REQ_CREDIT_BUREAU_HOUR",
#     "AMT_REQ_CREDIT_BUREAU_DAY",
#     "AMT_REQ_CREDIT_BUREAU_WEEK",
#     "AMT_REQ_CREDIT_BUREAU_MON",
#     "AMT_REQ_CREDIT_BUREAU_QRT",
#     "AMT_REQ_CREDIT_BUREAU_YEAR",
#     "AGE_INT",
#     "annuity_income_ratio",
#     "credit_annuity_ratio",
#     "credit_goods_price_ratio",
#     "credit_downpayment"
# ]


In [361]:
# data_dict = joblib.load("./bin/data_dict.joblib")
# ohe = joblib.load("./bin/ohe.joblib")
# categorical_imputer = joblib.load("./bin/categorical_imputer.joblib")
# simple_imputer = joblib.load("./bin/simple_imputer.joblib")
# scaler = joblib.load("./bin/scaler.joblib")
# model = joblib.load("./bin/model.joblib")

# #SimpleImputing (most frequent) and ohe of categorical features
# cat_array = categorical_imputer.transform(df_test[list_cat_features])
# cat_array = ohe.transform(cat_array).todense()

# #SimpleImputing (median) and StandardScaling of numerical features
# num_array = simple_imputer.transform(df_test[list_num_features])
# num_array = scaler.transform(num_array)

# #concatenate
# X = np.concatenate([cat_array, num_array], axis=1)
# X = np.asarray(X)

# #predict
# result_proba = model.predict_proba(X)
# y_pred_proba = result_proba[:,1]

# #cat columns list after ohe
# cat_features_list_after_ohe = ohe.get_feature_names(list_cat_features).tolist()

# #concatenate list of features
# features_list_after_prepr_test = cat_features_list_after_ohe + list_num_features

# #transform X into a dataframe with column names
# ohe_dataframe_test = pd.DataFrame(X, columns=features_list_after_prepr_test)



In [362]:
# sample_idx = 0
# sub_sampled_test_data = ohe_dataframe.iloc[sample_idx,:].values.reshape(1,-1)

In [363]:
# shap_vals = log_reg_explainer.shap_values(sub_sampled_test_data)

In [364]:
# print("Model prediction for test data", model.predict_proba(sub_sampled_test_data))

# shap.initjs()
# #https://shap-lrjball.readthedocs.io/en/latest/generated/shap.force_plot.html
# shap.force_plot(base_value=log_reg_explainer.expected_value[1],
#                 shap_values=shap_vals[1][0],
#                 features=sub_sampled_test_data[0],
#                 feature_names=features_list_after_prepr_test
# )

In [365]:
# #https://shap-lrjball.readthedocs.io/en/latest/generated/shap.summary_plot.html
# shap.summary_plot(shap_values=shap_vals, 
#                   features=sub_sampled_test_data, 
#                   feature_names=features_list_after_prepr_test, 
#                   max_display=10)

In [366]:
# shap.plots._waterfall.waterfall_legacy(log_reg_explainer.expected_value[1],#expected_value,
#                                        shap_vals[1][0],
#                                        sub_sampled_test_data[0],
#                                        feature_names=features_list_after_prepr_test,
#                                        max_display=10)

# Export DataSet

Export pre-processed dataset to be used for Dashboard Streamlit (on which heroku deployed model will predicting scoring) : 
- Sample 5% of the labelled dataset (app_train)
- Sample 5% of the unlabelled dataset (app_test)

In [367]:
#Labelled dataset

#recover Target label
app_train_init = pd.read_csv("../raw_data/application_train.csv").astype("object")
app_train["TARGET"] = app_train_init["TARGET"].astype(str)

#sample data
df_train = app_train.sample(frac=0.05, random_state=0)
df_train = df_train.astype('object')

#export
df_train.to_csv("./dashboard_data/df_train.csv", index=False)

print('df_train exported to dashboard_data folder.')

df_train exported to dashboard_data folder.


In [479]:
#Unlabelled dataset
app_test = pd.read_csv("../raw_data/application_test.csv").astype("object")

app_test_converted = convert_dtypes_func(app_test)
app_test_eng = feature_eng(app_test_converted)
app_test_selec_col = df_feature_selection(app_test_eng)

df_test = app_test_selec_col.sample(frac=0.05, random_state=0)
df_test = df_test.astype('object')

df_test.to_csv("./dashboard_data/df_test.csv", index=False)

print('df_test exported to dashboard_data folder.')


71 object columns were converted to int.
Feature engineering success
Feature selection success
df_test exported to dashboard_data folder.


# Serialization

In [393]:
joblib.dump(ohe, 'bin/ohe.joblib') #into a folder bin (for binary)
joblib.dump(categorical_imputer, 'bin/categorical_imputer.joblib')
joblib.dump(simple_imputer, 'bin/simple_imputer.joblib')
joblib.dump(scaler, 'bin/scaler.joblib')
joblib.dump(model, 'bin/model.joblib')

['bin/model.joblib']

# Pydantic Documentation

In [ ]:
# from pydantic import BaseModel, create_model, Field, ValidationError

# model = {}
# for i in range(len(app_train.dtypes)):
#     name = app_train.dtypes.index[i]
#     var_type = type(app_train.iloc[0, i])
#     model.update({name: (var_type, Field(...))})    

In [ ]:
# del model["SK_ID_CURR"]
# del model["TARGET"]
# # del model["DAYS_EMPLOYED_ANOM"]

In [ ]:
# data_dict = joblib.dump(model, 'bin/data_dict.joblib')

# DEBUG (TO DELETE)

In [480]:
df_test = pd.read_csv("./dashboard_data/df_test.csv")

list_cat_features = ["NAME_CONTRACT_TYPE",
    "CODE_GENDER",
    "FLAG_OWN_CAR",
    "FLAG_OWN_REALTY",
    "NAME_TYPE_SUITE",
    "NAME_INCOME_TYPE",
    "NAME_EDUCATION_TYPE",
    "NAME_FAMILY_STATUS",
    "NAME_HOUSING_TYPE",
    "FLAG_MOBIL",
    "FLAG_EMP_PHONE",
    "FLAG_WORK_PHONE",
    "FLAG_CONT_MOBILE",
    "FLAG_PHONE",
    "FLAG_EMAIL",
    "OCCUPATION_TYPE",
    "WEEKDAY_APPR_PROCESS_START",
    "REG_REGION_NOT_LIVE_REGION",
    "REG_REGION_NOT_WORK_REGION",
    "LIVE_REGION_NOT_WORK_REGION",
    "REG_CITY_NOT_LIVE_CITY",
    "REG_CITY_NOT_WORK_CITY",
    "LIVE_CITY_NOT_WORK_CITY",
    "ORGANIZATION_TYPE",
    "FONDKAPREMONT_MODE",
    "HOUSETYPE_MODE",
    "WALLSMATERIAL_MODE",
    "EMERGENCYSTATE_MODE",
    "FLAG_DOCUMENT_2",
    "FLAG_DOCUMENT_3",
    "FLAG_DOCUMENT_4",
    "FLAG_DOCUMENT_5",
    "FLAG_DOCUMENT_6",
    "FLAG_DOCUMENT_7",
    "FLAG_DOCUMENT_8",
    "FLAG_DOCUMENT_9",
    "FLAG_DOCUMENT_10",
    "FLAG_DOCUMENT_11",
    "FLAG_DOCUMENT_12",
    "FLAG_DOCUMENT_13",
    "FLAG_DOCUMENT_14",
    "FLAG_DOCUMENT_15",
    "FLAG_DOCUMENT_16",
    "FLAG_DOCUMENT_17",
    "FLAG_DOCUMENT_18",
    "FLAG_DOCUMENT_19",
    "FLAG_DOCUMENT_20",
    "FLAG_DOCUMENT_21"
]
list_num_features = [
    "CNT_CHILDREN",
    "AMT_INCOME_TOTAL",
    "AMT_CREDIT",
    "AMT_ANNUITY",
    "AMT_GOODS_PRICE",
    "DAYS_EMPLOYED",
    "DAYS_REGISTRATION",
    "DAYS_ID_PUBLISH",
    "REGION_RATING_CLIENT",
    "REGION_POPULATION_RELATIVE",
    "CNT_FAM_MEMBERS",
    "HOUR_APPR_PROCESS_START",
    "OWN_CAR_AGE",
    "EXT_SOURCE_1",
    "EXT_SOURCE_2",
    "EXT_SOURCE_3",
    "APARTMENTS_AVG",
    "BASEMENTAREA_AVG",
    "YEARS_BEGINEXPLUATATION_AVG",
    "YEARS_BUILD_AVG",
    "COMMONAREA_AVG",
    "ELEVATORS_AVG",
    "ENTRANCES_AVG",
    "FLOORSMAX_AVG",
    "FLOORSMIN_AVG",
    "LANDAREA_AVG",
    "LIVINGAPARTMENTS_AVG",
    "LIVINGAREA_AVG",
    "NONLIVINGAPARTMENTS_AVG",
    "NONLIVINGAREA_AVG",
    "APARTMENTS_MODE",
    "YEARS_BEGINEXPLUATATION_MODE",
    "FLOORSMIN_MODE",
    "LIVINGAREA_MODE",
    "LANDAREA_MEDI",
    "TOTALAREA_MODE",
    "OBS_30_CNT_SOCIAL_CIRCLE",
    "DEF_30_CNT_SOCIAL_CIRCLE",
    "DEF_60_CNT_SOCIAL_CIRCLE",
    "DAYS_LAST_PHONE_CHANGE",
    "AMT_REQ_CREDIT_BUREAU_HOUR",
    "AMT_REQ_CREDIT_BUREAU_DAY",
    "AMT_REQ_CREDIT_BUREAU_WEEK",
    "AMT_REQ_CREDIT_BUREAU_MON",
    "AMT_REQ_CREDIT_BUREAU_QRT",
    "AMT_REQ_CREDIT_BUREAU_YEAR",
    "AGE_INT",
    "annuity_income_ratio",
    "credit_annuity_ratio",
    "credit_goods_price_ratio",
    "credit_downpayment"
]


In [481]:
data_dict = joblib.load("./bin/data_dict.joblib")
ohe = joblib.load("./bin/ohe.joblib")
categorical_imputer = joblib.load("./bin/categorical_imputer.joblib")
simple_imputer = joblib.load("./bin/simple_imputer.joblib")
scaler = joblib.load("./bin/scaler.joblib")
model = joblib.load("./bin/model.joblib")

#SimpleImputing (most frequent) and ohe of categorical features
cat_array = categorical_imputer.transform(df_test[list_cat_features])
cat_array = ohe.transform(cat_array).todense()

#SimpleImputing (median) and StandardScaling of numerical features
num_array = simple_imputer.transform(df_test[list_num_features])
num_array = scaler.transform(num_array)

#concatenate
X = np.concatenate([cat_array, num_array], axis=1)
X = np.asarray(X)

#predict
result_proba = model.predict_proba(X)
y_pred_proba = result_proba[:,1]

df_test["pred"] = y_pred_proba
df_test["pred"] = round(df_test["pred"].astype(np.float64),4)




In [485]:
#cat columns list after ohe
cat_features_list_after_ohe = ohe.get_feature_names(list_cat_features).tolist()

#concatenate list of features
features_list_after_prepr_test = cat_features_list_after_ohe + list_num_features

#transform X into a dataframe with column names
ohe_dataframe_test = pd.DataFrame(X, columns=features_list_after_prepr_test)
ohe_dataframe_test["pred"] = y_pred_proba


In [482]:
df_test["pred"].value_counts()

0.00000    1992
0.64590       2
0.78840       2
0.48870       2
0.80480       2
           ... 
0.74710       1
0.51580       1
0.58120       1
0.59100       1
0.56160       1
Name: pred, Length: 429, dtype: int64

In [489]:
app_train.describe()

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,HOUR_APPR_PROCESS_START,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,YEARS_BEGINEXPLUATATION_MODE,FLOORSMIN_MODE,LIVINGAREA_MODE,LANDAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AGE_INT,annuity_income_ratio,credit_annuity_ratio,credit_goods_price_ratio,credit_downpayment,pred,pred_loaded
count,307511.00000,307511.00000,307511.00000,307499.00000,307233.00000,307511.00000,252137.00000,307511.00000,307511.00000,104582.00000,307509.00000,307511.00000,307511.00000,134133.00000,306851.00000,246546.00000,151450.00000,127568.00000,157504.00000,103023.00000,92646.00000,143620.00000,152683.00000,154491.00000,98869.00000,124921.00000,97312.00000,153161.00000,93997.00000,137829.00000,151450.00000,157504.00000,98869.00000,153161.00000,124921.00000,159080.00000,306490.00000,306490.00000,306490.00000,307510.00000,265992.00000,265992.00000,265992.00000,265992.00000,265992.00000,265992.00000,307511.00000,307499.00000,307499.00000,307233.00000,307233.00000,307511.00000,307511.00000
mean,0.41705,168797.91930,599025.99971,27108.57391,538396.20743,0.02087,-2384.16932,-4986.12033,-2994.20237,12.06109,2.15267,2.05246,12.06342,0.50213,0.51439,0.51085,0.11744,0.08844,0.97773,0.75247,0.04462,0.07894,0.14972,0.22628,0.23189,0.06633,0.10077,0.10740,0.00881,0.02836,0.11423,0.97707,0.22806,0.10598,0.06717,0.10255,1.42225,0.14342,0.10005,-962.85879,0.00640,0.00700,0.03436,0.26740,0.26547,1.89997,43.93698,7.35287,21.61225,1.12386,-60918.79680,0.57729,0.42271
std,0.72212,237123.14628,402490.77700,14493.73732,369446.46054,0.01383,2338.36016,3522.88632,1509.45042,11.94481,0.91068,0.50903,3.26583,0.21106,0.19106,0.19484,0.10824,0.08244,0.05922,0.11328,0.07604,0.13458,0.10005,0.14464,0.16138,0.08118,0.09258,0.11056,0.04773,0.06952,0.10794,0.06458,0.16116,0.11185,0.08217,0.10746,2.40099,0.44670,0.36229,826.80849,0.08385,0.11076,0.20468,0.91600,0.79406,1.86929,11.95613,9.44194,7.82362,0.12451,70495.11795,0.19817,0.19817
min,0.00000,25650.00000,45000.00000,1615.50000,40500.00000,0.00029,-17912.00000,-24672.00000,-7197.00000,0.00000,1.00000,1.00000,0.00000,0.01457,0.00000,0.00053,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,-4292.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,20.52000,0.53000,8.04000,0.15000,-540000.00000,0.00134,0.00000
25%,0.00000,112500.00000,270000.00000,16524.00000,238500.00000,0.01001,-3175.00000,-7479.50000,-4299.00000,5.00000,2.00000,2.00000,10.00000,0.33401,0.39246,0.37065,0.05770,0.04420,0.97670,0.68720,0.00780,0.00000,0.06900,0.16670,0.08330,0.01870,0.05040,0.04530,0.00000,0.00000,0.05250,0.97670,0.08330,0.04270,0.01870,0.04120,0.00000,0.00000,0.00000,-1570.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,34.01000,4.37000,15.61000,1.00000,-99972.00000,0.43492,0.26473
50%,0.00000,147150.00000,513531.00000,24903.00000,450000.00000,0.01885,-1648.00000,-4504.00000,-3254.00000,9.00000,2.00000,2.00000,12.00000,0.50600,0.56596,0.53528,0.08760,0.07630,0.98160,0.75520,0.02110,0.00000,0.13790,0.16670,0.20830,0.04810,0.07560,0.07450,0.00000,0.00360,0.08400,0.98160,0.20830,0.07310,0.04870,0.06880,0.00000,0.00000,0.00000,-757.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,43.15000,6.14000,20.00000,1.12000,-39204.00000,0.60033,0.399

In [490]:
app_train.columns

Index(['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER',
       'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'AMT_CREDIT', 'AMT_ANNUITY',
       ...
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
       'AMT_REQ_CREDIT_BUREAU_YEAR', 'AGE_INT', 'annuity_income_ratio',
       'credit_annuity_ratio', 'credit_goods_price_ratio',
       'credit_downpayment', 'pred', 'pred_loaded'],
      dtype='object', length=103)

In [483]:
df_test.describe()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,YEARS_BEGINEXPLUATATION_MODE,FLOORSMIN_MODE,LIVINGAREA_MODE,LANDAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AGE_INT,annuity_income_ratio,credit_annuity_ratio,credit_goods_price_ratio,credit_downpayment,pred
count,2437.00000,2437.00000,2437.00000,2437.00000,2435.00000,2437.00000,2437.00000,1992.00000,2437.00000,2437.00000,838.00000,2437.00000,2437.00000,2437.00000,2437.00000,2437.00000,2437.00000,2437.00000,2437.00000,2437.00000,2437.00000,2437.00000,2437.00000,2437.00000,2437.00000,2437.00000,1424.00000,2437.00000,1986.00000,1266.00000,1072.00000,1323.00000,858.00000,771.00000,1194.00000,1288.00000,1303.00000,818.00000,1055.00000,809.00000,1291.00000,776.00000,1150.00000,1266.00000,1323.00000,818.00000,1291.00000,1055.00000,1332.00000,2435.00000,2435.00000,2435.00000,2437.00000,2437.00000,2437.00000,2437.00000,2437.00000,2437.00000,2437.00000,2437.00000,2437.00000,2437.00000,2437.00000,2437.00000,2437.00000,2437.00000,2437.00000,2437.00000,2437.00000,2437.00000,2437.00000,2437.00000,2437.00000,2125.00000,2125.00000,2125.00000,2125.00000,2125.00000,2125.00000,2437.00000,2435.00000,2435.00000,2437.00000,2437.00000,2437.00000
mean,280379.60854,0.37464,180298.98872,516991.74928,29741.67351,464163.87464,0.02121,-2553.78815,-4924.96758,-3028.98892,11.35322,1.00000,0.81740,0.21297,0.99754,0.24949,0.15388,2.12556,2.04144,12.00369,0.02134,0.06360,0.04760,0.07961,0.22651,0.17357,0.49763,0.52213,0.49733,0.12055,0.09077,0.98021,0.74414,0.04764,0.08972,0.14739,0.23007,0.23748,0.06551,0.10680,0.11069,0.01198,0.02887,0.11677,0.97999,0.23171,0.10907,0.06623,0.10617,1.45092,0.14990,0.10719,-1070.69922,0.00000,0.79565,0.00000,0.01067,0.08289,0.00000,0.09110,0.00451,0.00000,0.00123,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00123,0.00000,0.00000,0.00000,0.00235,0.00188,0.00282,0.00800,0.56094,1.99718,44.21022,7.13181,17.07612,1.13146,-52827.87464,0.10902
std,103418.47277,0.70038,111900.67572,364875.22338,16546.03300,338666.92254,0.01450,2460.82434,3610.51022,1562.40508,9.93019,0.00000,0.38642,0.40949,0.04957,0.43280,0.36091,0.87606,0.52554,3.29034,0.14454,0.24409,0.21296,0.27074,0.41866,0.37882,0.20253,0.17789,0.18921,0.11642,0.09337,0.02851,0.12694,0.08451,0.14687,0.10101,0.14563,0.16334,0.08168,0.10972,0.11595,0.06068,0.06424,0.11647,0.02856,0.16265,0.11881,0.08306,0.11404,2.30544,0.44514,0.36767,888.13304,0.00000,0.40331,0.00000,0.10276,0.27577,0.00000,0.28780,0.06705,0.00000,0.03507,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.03507,0.00000,0.00000,0.00000,0.04846,0.05312,0.05307,0.09424,0.70489,1.89880,11.64006,4.69407,6.32433,0.12442,60584.62511,0.24371
min,100169.00000,0.00000,30150.00000,45000.00000,2965.50000,45000.00000,0.00

In [486]:
ohe_dataframe_test.describe()

,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_F,CODE_GENDER_M,CODE_GENDER_XNA,FLAG_OWN_CAR_N,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_N,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,FLAG_MOBIL_0,FLAG_MOBIL_1,FLAG_EMP_PHONE_0,FLAG_EMP_PHONE_1,FLAG_WORK_PHONE_0,FLAG_WORK_PHONE_1,FLAG_CONT_MOBILE_0,FLAG_CONT_MOBILE_1,FLAG_PHONE_0,FLAG_PHONE_1,FLAG_EMAIL_0,FLAG_EMAIL_1,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Waiters/barmen staff,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,REG_REGION_NOT_LIVE_REGION_0,REG_REGION_NOT_LIVE_REGION_1,REG_REGION_NOT_WORK_REGION_0,REG_REGION_NOT_WORK_REGION_1,LIVE_REGION_NOT_WORK_REGION_0,LIVE_REGION_NOT_WORK_REGION_1,REG_CITY_NOT_LIVE_CITY_0,REG_CITY_NOT_LIVE_CITY_1,REG_CITY_NOT_WORK_CITY_0,REG_CITY_NOT_WORK_CITY_1,LIVE_CITY_NOT_WORK_CITY_0,LIVE_CITY_NOT_WORK_CITY_1,ORGANIZATION_TYPE_Advertising,ORGANIZATION_TYPE_Agriculture,ORGANIZATION_TYPE_Bank,ORGANIZATION_TYPE_Business Entity Type 1,ORGANIZATION_TYPE_Business Entity Type 2,ORGANIZATION_TYPE_Business Entity Type 3,ORGANIZATION_TYPE_Cleaning,ORGANIZATION_TYPE_Construction,ORGANIZATION_TYPE_Culture,ORGANIZATION_TYPE_Electricity,ORGANIZATION_TYPE_Emergency,ORGANIZATION_TYPE_Government,ORGANIZATION_TYPE_Hotel,ORGANIZATION_TYPE_Housing,ORGANIZATION_TYPE_Industry: type 1,ORGANIZATION_TYPE_Industry: type 10,ORGANIZATION_TYPE_Industry: type 11,ORGANIZATION_TYPE_Industry: type 12,ORGANIZATION_TYPE_Industry: type 13,ORGANIZATION_TYPE_Industry: type 2,ORGANIZATION_TYPE_Industry: type 3,ORGANIZATION_TYPE_Industry: type 4,ORGANIZATION_TYPE_Industry: type 5,ORGANIZATION_TYPE_Industry: type 6,ORGANIZATION_TYPE_Industry: type 7,ORGANIZATION_TYPE_Industry: type 8,ORGANIZATION_TYPE_Industry: type 9,ORGANIZATION_TYPE_Insurance,ORGANIZATION_TYPE_Kindergarten,ORGANIZATION_TYPE_Legal Services,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Military,ORGANIZATION_TYPE_Mobile,ORGANIZATION_TYPE_Other,ORGANIZATION_TYPE_Police,ORGANIZATION_TYPE_Postal,ORGANIZATION_TYPE_Realtor,ORGANIZATION_TYPE_Religion,ORGANIZATION_TYPE_Restaurant,ORGANIZATION_TYPE_School,ORGANIZATION_TYPE_Security,ORGANIZATION_TYPE_Security Ministries,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Services,ORGANIZATION_TYPE_Telecom,ORGANIZATION_TYPE_Trade: type 1,ORGANIZATION_TYPE_Trade: type 2,ORGANIZATION_TYPE_Trade: type 3,ORGANIZATION_TYPE_Trade: typ

In [419]:
df_test.shape[0] - 1992

445

In [408]:
df_test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,YEARS_BEGINEXPLUATATION_MODE,FLOORSMIN_MODE,LIVINGAREA_MODE,LANDAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AGE_INT,annuity_income_ratio,credit_annuity_ratio,credit_goods_price_ratio,credit_downpayment,pred
0,223197,Cash loans,F,N,Y,0.0,117000.0,630000.0,32296.5,630000.0,"Spouse, partner",Working,Higher education,Married,With parents,0.028663,-3692.0,-8012.0,-4939.0,NaN,1,1,0,1,0,1,Core staff,2.0,2.0,WEDNESDAY,9.0,0,0,0,0,0,0,Kindergarten,NaN,0.466105,0.411849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0,38.14,3.62,19.51,1.0,0.0,0.0
1,135591,Cash loans,F,N,Y,0.0,202500.0,675000.0,36747.0,675000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.01452,-1951.0,-6206.0,-1607.0,NaN,1,1,0,1,1,1,Sales staff,2.0,2.0,SATURDAY,9.0,0,0,0,0,1,1,Business Entity Type 3,NaN,0.260932,0.533482,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,-306.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,2.0,50.61,5.51,18.37,1.0,0.0,0.0
2,334756,Cash loans,F,Y,Y,1.0,121500.0,450000.0,23107.5,450000.0,NaN,Commercial associate,Higher education,Married,House / apartment,0.018209,-3939.0,-4367.0,-4367.0,23.0,1,1,0,1,0,1,Accountants,3.0,3.0,MONDAY,7.0,0,0,0,0,0,0,Business Entity Type 3,0.514785,0.354315,0.251239,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0,1.0,-2581.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,45.02,5.26,19.47,1.0,0.0,0.0
3,212448,Cash loans,M,Y,Y,0.0,225000.0,765000.0,39060.0,765000.0,Unaccompanied,Commercial associate,Higher education,Married,House / apartment,0.00702,-1460.0,-461.0,-4777.0,9.0,1,1,1,1,0,0,Core staff,2.0,2.0,TUESDAY,9.0,0,0,0,0,0,0,Self-employed,NaN,0.575978,0.683269,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,-1660.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,6.0,44.96,5.76,19.59,1.0,0.0,0.0
4,157306,Cash loans,F,N,N,0.0,76500.0,315000.0,17086.5,315000.0,Unaccompanied,Working,Incomplete higher,Single / not married,House / apartment,0.008625,-789.0,-3529.0,-1682.0,NaN,1,1,1,1,0,0,NaN,1.0,2.0,SATURDAY,13.0,0,0,0,0,0,0,Business Entity Type 2,0.368835,0.311492

In [372]:
df_test[df_test["SK_ID_CURR"] == 334756]

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,YEARS_BEGINEXPLUATATION_MODE,FLOORSMIN_MODE,LIVINGAREA_MODE,LANDAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AGE_INT,annuity_income_ratio,credit_annuity_ratio,credit_goods_price_ratio,credit_downpayment
2,334756,Cash loans,F,Y,Y,1.0,121500.0,450000.0,23107.5,450000.0,NaN,Commercial associate,Higher education,Married,House / apartment,0.018209,-3939.0,-4367.0,-4367.0,23.0,1,1,0,1,0,1,Accountants,3.0,3.0,MONDAY,7.0,0,0,0,0,0,0,Business Entity Type 3,0.514785,0.354315,0.251239,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0,1.0,-2581.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,45.02,5.26,19.47,1.0,0.0


In [375]:
model.predict_proba(X).shape

(2437, 2)

In [463]:
ohe_dataframe_test["DAYS_EMPLOYED"].describe()

count       2437.00000
mean     -150924.80635
std       175962.25449
min     -1178130.09487
25%      -214587.92445
50%       -95726.71576
75%       -23571.31049
max           -0.14591
Name: DAYS_EMPLOYED, dtype: float64